# Factorisation LU

## Introduction

La décomposition $LU$ est une factorisation fondamentale en algèbre linéaire numérique qui exprime une matrice carrée $A$ comme produit d'une matrice triangulaire inférieure $L$ et d'une matrice triangulaire supérieure $U$ :

$$
A = LU.
$$

C'est un outil clé pour la résolution efficace de systèmes linéaires $Ax=b$, après factorisation, chaque résolution se réduit à deux substitutions triangulaires (avant et arrière), beaucoup moins coûteuses qu'une inversion complète.

La factorisation $LU$ reflète formellement le procédé d'élimination de Gauss.

Remarque importante : dans sa forme la plus simple (sans permutations), la factorisation $A=LU$ existe lorsque les pivots de Gauss rencontrés lors de l'élimination sont non nuls. Si un pivot est nul (ou numériquement trop petit), il faut recourir à une factorisation avec permutation (pivot partiel) sous la forme

$$
PA = LU,
$$

où $P$ est une matrice de permutation. Dans cette section, nous considérons d'abord le cas simple sans permutations.

## Définition et propriété

### Définition

Soit $A \in M_n(\mathbb{R})$ une matrice carrée inversible.  
On dit que $A$ admet une **décomposition LU** si elle peut s’écrire sous la forme :

$$
A = LU
$$

où :

- $L$ est une matrice **triangulaire inférieure** avec des coefficients diagonaux égaux à $1$ ;
- $U$ est une matrice **triangulaire supérieure**.

Autrement dit, la décomposition LU sépare les opérations d’élimination de Gauss en deux étapes :  
$L$ contient les multiplicateurs utilisés pour annuler les coefficients sous la diagonale, et $U$ contient les coefficients restants après l’élimination.

### Proposition 


Soit une matrice  

$$
A = (a_{ij})_{1 \le i,j \le n}
$$  

d’ordre $n$ telle que toutes les sous-matrices diagonales d’ordre $k$, définies par 

$$
\Delta^k =
\begin{pmatrix}
a_{11} & \cdots & a_{1k} \\
\vdots & \ddots & \vdots \\
a_{k1} & \cdots & a_{kk}
\end{pmatrix},
$$

soient inversibles.  
Il existe alors un unique couple de matrices $(L, U)$, avec $U$ triangulaire supérieure, et $L$ triangulaire inférieure ayant une diagonale de 1, tel que  

$$
A = LU.
$$

## Algorithme

L’objectif est de déterminer deux matrices triangulaires $L$ et $U$ telles que :

$$
A = LU,
$$

où $L$ est triangulaire inférieure avec des $1$ sur la diagonale, et $U$ est triangulaire supérieure.

L’algorithme utilisé repose sur le **principe de l’élimination de Gauss** sans pivot.


### Étapes de l’algorithme

Soit $A = (a_{ij})$ une matrice carrée d’ordre $n$.

1. **Initialisation :**
   - On définit $L = I_n$ (matrice identité).
   - On définit $U$ comme une matrice nulle de même dimension que $A$.

2. **Pour chaque colonne $j = 0, 1, \dots, n-1$ :**

   - **Calcul des éléments de la ligne $j$ de $U$ :**

     $$
     U_{ij} = a_{ij} - \sum_{k=0}^{i-1} L_{ik} U_{kj}, \quad \text{pour } i \le j
     $$

   - **Calcul des éléments de la colonne $j$ de $L$ :**

     $$
     L_{ij} = \frac{1}{U_{jj}} \left(a_{ij} - \sum_{k=0}^{j-1} L_{ik} U_{kj}\right), \quad \text{pour } i > j
     $$

3. **À la fin de la procédure**, on obtient deux matrices $L$ et $U$ telles que $A = LU$.


### Remarque

L’algorithme suppose que **tous les pivots $U_{jj}$ sont non nuls**.  
S’il arrive qu’un pivot soit nul, la décomposition $LU$ ne peut pas être calculée sans **pivotement** (échange de lignes).  
Ce cas correspond à la version généralisée de la décomposition : $PA = LU$, où $P$ est une matrice de permutation.



### Implémentation et vérification

In [11]:
import numpy as np
from scipy.linalg import lu

In [25]:
def LU_Factorization(A):
    n = A.shape[0]
    L=np.eye(n)
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j+1):
            somme = 0
            for k in range(i):
                somme+=L[i,k]*U[k,j]
            U[i,j]=A[i,j] - somme
        for i in range(j+1,n):
            somme = 0
            for k in range(j):
                somme += L[i,k]*U[k,j]
            L[i,j]=(A[i,j]-somme)/U[j,j]
    return L,U

In [10]:
A = np.array([
    [2, -1, -2],
    [-4, 6, 3],
    [-4, -2, 8]
], dtype=float)

In [13]:
L_custom, U_custom = LU_Factorization(A)


print("A :\n", A)

print("\n décomposition LU :")
print("L_custom :\n", L_custom)
print("U_custom :\n", U_custom)
print("Vérif A ≈ L_custom @ U_custom :",  L_custom @ U_custom)



A :
 [[ 2. -1. -2.]
 [-4.  6.  3.]
 [-4. -2.  8.]]

 décomposition LU :
L_custom :
 [[ 1.  0.  0.]
 [-2.  1.  0.]
 [-2. -1.  1.]]
U_custom :
 [[ 2. -1. -2.]
 [ 0.  4. -1.]
 [ 0.  0.  3.]]
Vérif A ≈ L_custom @ U_custom : [[ 2. -1. -2.]
 [-4.  6.  3.]
 [-4. -2.  8.]]


## Application à la nésolution de l'équation linéaire Ax = b

Soit $A \in M_n(\mathbb{R})$ une matrice carrée inversible et $b \in \mathbb{R}^n$.  
On souhaite résoudre le système :

$$
A x = b.
$$


### Utilisation de la décomposition $LU$

Si $A$ admet une décomposition $A = LU$, alors :

$$
LU x = b.
$$

On pose :
$$
U x = y.
$$

Le système s’écrit alors sous la forme :

$$
L y = b \quad \text{(1)}
$$
$$
U x = y \quad \text{(2)}
$$

On résout donc le système $Ax = b$ en deux étapes successives :  
- une **substitution avant** pour résoudre $(1)$ ;  
- une **substitution arrière** pour résoudre $(2)$.

### Substitution avant (résolution de $L y = b$)



Comme $L$ est triangulaire inférieure avec des $1$ sur la diagonale, on peut calculer les composantes de $y$ successivement :

$$
y_i = b_i - \sum_{k=0}^{i-1} L_{ik} y_k, \quad \text{pour } i = 0, 1, \dots, n-1.
$$


### Substitution arrière (résolution de $U x = y$)

Puisque $U$ est triangulaire supérieure, on calcule les composantes de $x$ de bas en haut :

$$
x_i = \frac{1}{U_{ii}} \left( y_i - \sum_{k=i+1}^{n-1} U_{ik} x_k \right), \quad \text{pour } i = n-1, \dots, 0.
$$


### Implémentation

#### Résolution de $L y = b$

In [15]:
def resolve_L(L,b):
    n=L.shape[0]
    y=np.zeros(n)
    for i in range(n):
        somme=0
        for k in range(i):
            somme+=L[i,k]*y[k]
        y[i]=b[i]-somme
    return y
            
            
          
    

#### Vérification

In [24]:
L = np.array([
    [1, 0, 0],
    [3, 1, 0],
    [2, -1, 1]
], dtype=float)

b = np.array([5, 11, 4], dtype=float)

y = resolve_L(L, b)
print("y =", y)
print("Vérification Lx ≈ y :", np.allclose(L @ y, b))  # doit être True

y = [  5.  -4. -10.]
Vérification Lx ≈ y : True


#### Résolution de $U x = y$

In [21]:
def resolve_U(U,y):
    n=U.shape[0]
    x=np.zeros(n)
    for i in range(n-1,-1,-1):
        somme=0
        for k in range(i+1,n):
            somme+=U[i,k]*x[k]
        x[i]=(y[i]-somme)/U[i,i]
    return x
    

#### Vérification

In [22]:
U = np.array([
    [3, -2, 1],
    [0, 5, 2],
    [0, 0, 4]
], dtype=float)

y = np.array([4, 3, 8], dtype=float)

x = resolve_U(U, y)
print("x =", x)
print("Vérification Ux ≈ y :", np.allclose(U @ x, y))  # doit être True

x = [ 0.53333333 -0.2         2.        ]
Vérification Ux ≈ y : True


#### Résolution $Ax = b$

In [27]:
def solve_LU(A,b):
    L,U=LU_Factorization(A)
    y=resolve_L(L,b)
    x=resolve_U(U,y)
    return x



#### Vérification

In [28]:
A = np.array([
    [2, -1, -2],
    [-4, 6, 3],
    [-4, -2, 8]
], dtype=float)

b = np.array([-2, 9, -5], dtype=float)

x = solve_LU(A, b)

print("x =", x)
print("Vérification A·x ≈ b :", np.allclose(A @ x, b))  # doit être True

x = [-1.875       0.91666667 -1.33333333]
Vérification A·x ≈ b : True
